In [1]:
import os

In [2]:
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification\\notebooks'

In [3]:
os.chdir("../")
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification'

In [6]:
import yaml
yaml_file = "config\config.yaml"
with open(yaml_file) as file:
    content = yaml.safe_load(file)
    print(content)

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_path': 'D:/Deep_Learning_Projects/Sports_Classification/notebooks/data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}


In [13]:
config = content["data_ingestion"]

In [15]:
config['root_dir']

'artifacts/data_ingestion'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_path : Path
    local_data_file : Path
    unzip_dir : Path

In [22]:
from src.Sports_Classification.Constants import *
from src.Sports_Classification.utils.utils import read_yaml, create_directories

In [33]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']

        create_directories([self.config['data_ingestion']['root_dir']])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_path=config['source_path'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )

        return data_ingestion_config

In [38]:
import os, zipfile
from src.Sports_Classification.logger import logging
from src.Sports_Classification.exception import CustomException
import urllib.request as request
from src.Sports_Classification.utils.utils import *

class DataIngestion:

    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        The function will download the files from the given url

        source_path (url) : The url for the file to be downloaded
        local_data_file (path) : The path where the file should be downloaded

        """
        if not (os.path.exists(self.config['local_data_file'])):
            filename, headers = request.urlretrieve(
                url=self.config['source_path'],
                filename=self.config['local_data_file']
            )

            logging.info(f"Download the {filename} with the info: \n{headers}")
    
        else:
            logging.info(f"the file already exists of size: {get_size(Path(self.config['local_data_file']))}")

    
    def extract_zip_file(self):
        """
        The function unzip the downloaded zip file

        unzip_dir (path) : The path where the zip file should be unzipped
        """
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_dir)


In [40]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise CustomException(e, sys)

{'root_dir': 'artifacts/data_ingestion', 'source_path': 'D:/Deep_Learning_Projects/Sports_Classification/notebooks/data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_path='D:/Deep_Learning_Projects/Sports_Classification/notebooks/data.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
